In [1]:
# Análise de Média de Pessoas em Aglomerações por Período
# Base: Planilha processada com campos parseados
# Data início: Informada pelo usuário
# Data fim: Informada pelo usuário

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Definir caminho da raiz do projeto
# Funciona tanto como script .py quanto em notebook
if '__file__' in globals():
    # Executando como script .py
    projeto_root = Path(__file__).parent.parent.parent
else:
    # Executando como notebook ou célula
    projeto_root = Path.cwd()
    # Se estiver em notebooks/, subir um nível
    if projeto_root.name == 'notebooks':
        projeto_root = projeto_root.parent

print("=" * 80)
print("ANÁLISE: Média de Pessoas em Aglomerações por Período")
print("=" * 80)
print(f"✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Caminhos

# %%
PROCESSED_DIR = projeto_root / 'data' / 'processed'
DOCS_DIR = projeto_root / 'docs'

# Garantir que o diretório docs existe
DOCS_DIR.mkdir(exist_ok=True)

print(f"\n✓ Diretório base: {projeto_root}")
print(f"✓ Planilhas processadas: {PROCESSED_DIR}")
print(f"✓ Saída de documentos: {DOCS_DIR}")

# %% [markdown]
# # 3. Listar e Selecionar Planilha Processada

# %%
# Listar arquivos .xlsx na pasta processed
arquivos_disponiveis = sorted(PROCESSED_DIR.glob('*.xlsx'))

if not arquivos_disponiveis:
    print("\n⚠️  ERRO: Nenhum arquivo .xlsx encontrado na pasta 'data/processed/'")
    print("   Certifique-se de que existem planilhas processadas no diretório.")
    exit()

print(f"\n📁 Arquivos disponíveis em 'data/processed/':")
print("-" * 80)
for idx, arquivo in enumerate(arquivos_disponiveis, 1):
    print(f"  [{idx}] {arquivo.name}")
print("-" * 80)

# Solicitar seleção do usuário
while True:
    try:
        selecao = input(f"\nSelecione o número do arquivo [1-{len(arquivos_disponiveis)}]: ").strip()
        idx_selecionado = int(selecao) - 1
        
        if 0 <= idx_selecionado < len(arquivos_disponiveis):
            arquivo_selecionado = arquivos_disponiveis[idx_selecionado]
            print(f"\n✓ Arquivo selecionado: {arquivo_selecionado.name}")
            break
        else:
            print(f"⚠️  Por favor, escolha um número entre 1 e {len(arquivos_disponiveis)}")
    except ValueError:
        print("⚠️  Entrada inválida. Digite apenas o número do arquivo.")
    except KeyboardInterrupt:
        print("\n\n⚠️  Operação cancelada pelo usuário.")
        exit()

# %% [markdown]
# # 4. Solicitar Datas de Início e Fim da Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DE ANÁLISE")
print("=" * 80)

# Solicitar data de início
while True:
    try:
        data_inicio_input = input("\n📅 Digite a data INÍCIO da análise (formato: DD/MM/AAAA): ").strip()
        data_inicio = datetime.strptime(data_inicio_input, '%d/%m/%Y')
        print(f"✓ Data início definida: {data_inicio.strftime('%d/%m/%Y')}")
        break
        
    except ValueError:
        print("⚠️  Formato inválido. Use DD/MM/AAAA (exemplo: 01/05/2025)")
    except KeyboardInterrupt:
        print("\n\n⚠️  Operação cancelada pelo usuário.")
        exit()

# Solicitar data de fim
while True:
    try:
        data_fim_input = input("\n📅 Digite a data FIM da análise (formato: DD/MM/AAAA): ").strip()
        data_fim = datetime.strptime(data_fim_input, '%d/%m/%Y')
        
        # Validar se a data fim é posterior à data início
        if data_fim < data_inicio:
            print(f"⚠️  A data fim deve ser posterior à data início ({data_inicio.strftime('%d/%m/%Y')})")
            continue
        
        # Validar se as datas não são iguais
        if data_fim == data_inicio:
            print("⚠️  A data fim deve ser diferente da data início")
            continue
        
        print(f"✓ Data fim definida: {data_fim.strftime('%d/%m/%Y')}")
        print(f"\n✓ Período de análise: {data_inicio.strftime('%d/%m/%Y')} até {data_fim.strftime('%d/%m/%Y')}")
        
        # Calcular e mostrar duração
        dias_totais = (data_fim - data_inicio).days + 1
        print(f"✓ Duração: {dias_totais} dias")
        break
        
    except ValueError:
        print("⚠️  Formato inválido. Use DD/MM/AAAA (exemplo: 20/10/2025)")
    except KeyboardInterrupt:
        print("\n\n⚠️  Operação cancelada pelo usuário.")
        exit()

# %% [markdown]
# # 5. Carregar e Validar Dados

# %%
print(f"\n📊 Carregando dados de: {arquivo_selecionado.name}")
print("-" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Planilha carregada com sucesso!")
    print(f"✓ Total de registros: {len(df):,}")
    
except Exception as e:
    print(f"\n⚠️  ERRO ao carregar planilha: {e}")
    exit()

# %% [markdown]
# # 6. Validar Estrutura Esperada

# %%
colunas_esperadas = [
    'Equipe', 'Data', 'Logradouro', 'Período', 'Qtd. pessoas',
    'tipo_logradouro', 'nome_logradouro', 'numero_logradouro', 
    'complemento_logradouro'
]

colunas_faltantes = [col for col in colunas_esperadas if col not in df.columns]

if colunas_faltantes:
    print(f"\n⚠️  ERRO: Colunas obrigatórias não encontradas:")
    for col in colunas_faltantes:
        print(f"   - {col}")
    print("\n   Certifique-se de que está usando uma planilha processada.")
    exit()

print(f"\n✓ Estrutura da planilha validada!")

# %% [markdown]
# # 7. Converter Campo Data e Filtrar Período

# %%
# Converter coluna Data para datetime
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Verificar conversão
datas_invalidas = df['Data'].isna().sum()
if datas_invalidas > 0:
    print(f"\n⚠️  Atenção: {datas_invalidas} registros com datas inválidas (serão removidos)")
    df = df.dropna(subset=['Data'])

# Extrair mês e ano
df['Mes'] = df['Data'].dt.month
df['Ano'] = df['Data'].dt.year

print(f"\n✓ Conversão de datas concluída")
print(f"✓ Período dos dados: {df['Data'].min().strftime('%d/%m/%Y')} até {df['Data'].max().strftime('%d/%m/%Y')}")

# Filtrar dados de acordo com o período definido
df_filtrado = df[(df['Data'] >= data_inicio) & (df['Data'] <= data_fim)].copy()

print(f"\n🔍 Filtrando dados:")
print("-" * 80)
print(f"✓ Data início: {data_inicio.strftime('%d/%m/%Y')}")
print(f"✓ Data fim: {data_fim.strftime('%d/%m/%Y')}")
print(f"✓ Total de registros após filtro: {len(df_filtrado):,}")

if len(df_filtrado) == 0:
    print(f"\n⚠️  ERRO: Nenhum registro encontrado para o período especificado.")
    print(f"   Período solicitado: {data_inicio.strftime('%d/%m/%Y')} a {data_fim.strftime('%d/%m/%Y')}")
    print(f"   Período disponível nos dados: {df['Data'].min().strftime('%d/%m/%Y')} a {df['Data'].max().strftime('%d/%m/%Y')}")
    exit()

# %% [markdown]
# # 8. Garantir Tipo Numérico

# %%
# Garantir que Qtd. pessoas é numérico
df_filtrado['Qtd. pessoas'] = pd.to_numeric(df_filtrado['Qtd. pessoas'], errors='coerce')
df_filtrado = df_filtrado.dropna(subset=['Qtd. pessoas'])

print(f"\n✓ Total de registros com Qtd. pessoas válida: {len(df_filtrado):,}")

# %% [markdown]
# # 9. Mapear Períodos

# %%
# Mapear períodos para categorias
mapeamento_periodo = {
    '05h - Madrugada': 'Madrugada',
    '10h - Manhã': 'Manhã',
    '15h - Tarde': 'Tarde',
    '20h - Noite': 'Noite'
}

df_filtrado['Periodo_Categoria'] = df_filtrado['Período'].map(mapeamento_periodo)

# Verificar mapeamento
periodos_nao_mapeados = df_filtrado['Periodo_Categoria'].isna().sum()
if periodos_nao_mapeados > 0:
    print(f"\n⚠️  Atenção: {periodos_nao_mapeados} registros com períodos não mapeados")
else:
    print(f"\n✓ Todos os períodos mapeados com sucesso!")

# Remover registros sem período mapeado
df_filtrado = df_filtrado.dropna(subset=['Periodo_Categoria'])

# %% [markdown]
# # 10. Filtrar Aglomerações (> 10 pessoas)

# %%
df_aglomeracoes = df_filtrado[df_filtrado['Qtd. pessoas'] > 10].copy()

print(f"\n👥 Filtrando aglomerações (> 10 pessoas):")
print("-" * 80)
print(f"✓ Total de registros no período: {len(df_filtrado):,}")
print(f"✓ Total de aglomerações (>10 pessoas): {len(df_aglomeracoes):,}")

if len(df_filtrado) > 0:
    print(f"✓ Percentual de aglomerações: {(len(df_aglomeracoes) / len(df_filtrado) * 100):.1f}%")

# %% [markdown]
# # 11. Calcular Dias por Mês

# %%
# Calcular dias únicos por mês
dias_por_mes = df_filtrado.groupby('Mes')['Data'].nunique().to_dict()

nomes_meses = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

print(f"\n📅 Quantidade de dias em cada mês:")
print("-" * 80)
meses_unicos = sorted(df_filtrado['Mes'].unique())
for mes in meses_unicos:
    dias = dias_por_mes.get(mes, 0)
    print(f"  • {nomes_meses[mes]}: {dias} dias")

# %% [markdown]
# # 12. Calcular Agregações por Mês e Período

# %%
print(f"\n📈 Calculando agregações:")
print("-" * 80)

periodos = ['Madrugada', 'Manhã', 'Tarde', 'Noite']

# Agregações gerais (todas as pessoas)
agg_geral = df_filtrado.groupby(['Mes', 'Periodo_Categoria']).agg({
    'Qtd. pessoas': ['sum', 'count']
}).reset_index()
agg_geral.columns = ['Mes', 'Periodo_Categoria', 'Soma_Pessoas', 'Qtd_Registros']

# Agregações de aglomerações (apenas >10)
agg_aglom = df_aglomeracoes.groupby(['Mes', 'Periodo_Categoria']).agg({
    'Qtd. pessoas': ['sum', 'count']
}).reset_index()
agg_aglom.columns = ['Mes', 'Periodo_Categoria', 'Soma_Pessoas_Aglom', 'Qtd_Aglomeracoes']

# Merge
dados_completos = pd.merge(agg_geral, agg_aglom, on=['Mes', 'Periodo_Categoria'], how='left')
dados_completos['Qtd_Aglomeracoes'] = dados_completos['Qtd_Aglomeracoes'].fillna(0).astype(int)
dados_completos['Soma_Pessoas_Aglom'] = dados_completos['Soma_Pessoas_Aglom'].fillna(0)

print(f"✓ Agregações concluídas!")

# %% [markdown]
# # 13. Criar Aba Geral

# %%
print(f"\n📊 Criando aba geral...")

linhas_geral = []

for mes in meses_unicos:
    dias = dias_por_mes.get(mes, 0)
    
    # Somar todos os períodos do mês
    dados_mes = dados_completos[dados_completos['Mes'] == mes]
    
    soma_pessoas = dados_mes['Soma_Pessoas'].sum()
    qtd_aglomeracoes = dados_mes['Qtd_Aglomeracoes'].sum()
    soma_pessoas_aglom = dados_mes['Soma_Pessoas_Aglom'].sum()
    
    # Calcular médias
    # Média de pessoas: soma total / (dias * 4 períodos)
    # Pois cada dia tem 4 contagens (Madrugada, Manhã, Tarde, Noite)
    total_contagens = dias * 4
    media_pessoas = soma_pessoas / total_contagens if total_contagens > 0 else 0
    media_pessoas_aglom = soma_pessoas_aglom / total_contagens if total_contagens > 0 else 0
    
    linhas_geral.append({
        'Mês': nomes_meses[mes],
        'Qtd. Dias': dias,
        'Soma Pessoas': int(soma_pessoas),
        'Média Pessoas': round(media_pessoas, 2),
        'Qtd. Aglomerações': int(qtd_aglomeracoes),
        'Soma Pessoas Aglomerações': int(soma_pessoas_aglom),
        'Média Pessoas Aglomerações': round(media_pessoas_aglom, 2)
    })

df_geral = pd.DataFrame(linhas_geral)
print(f"✓ Aba geral criada com {len(df_geral)} linhas")

# %% [markdown]
# # 14. Criar Abas por Período

# %%
print(f"\n📊 Criando abas por período...")

dfs_periodos = {}

for periodo in periodos:
    linhas_periodo = []
    
    for mes in meses_unicos:
        dias = dias_por_mes.get(mes, 0)
        
        # Filtrar dados do período específico
        dados_periodo = dados_completos[
            (dados_completos['Mes'] == mes) & 
            (dados_completos['Periodo_Categoria'] == periodo)
        ]
        
        if len(dados_periodo) > 0:
            soma_pessoas = dados_periodo['Soma_Pessoas'].values[0]
            qtd_aglomeracoes = dados_periodo['Qtd_Aglomeracoes'].values[0]
            soma_pessoas_aglom = dados_periodo['Soma_Pessoas_Aglom'].values[0]
        else:
            soma_pessoas = 0
            qtd_aglomeracoes = 0
            soma_pessoas_aglom = 0
        
        # Calcular médias
        media_pessoas = soma_pessoas / dias if dias > 0 else 0
        media_pessoas_aglom = soma_pessoas_aglom / dias if dias > 0 else 0
        
        linhas_periodo.append({
            'Mês': nomes_meses[mes],
            'Qtd. Dias': dias,
            'Soma Pessoas': int(soma_pessoas),
            'Média Pessoas': round(media_pessoas, 2),
            'Qtd. Aglomerações': int(qtd_aglomeracoes),
            'Soma Pessoas Aglomerações': int(soma_pessoas_aglom),
            'Média Pessoas Aglomerações': round(media_pessoas_aglom, 2)
        })
    
    dfs_periodos[periodo] = pd.DataFrame(linhas_periodo)
    print(f"✓ Aba '{periodo}' criada com {len(linhas_periodo)} linhas")

# %% [markdown]
# # 15. Criar Aba para Gráfico

# %%
print(f"\n📊 Criando aba para gráfico...")

linhas_grafico = []

for mes in meses_unicos:
    dias = dias_por_mes.get(mes, 0)
    linha = {'Mês': nomes_meses[mes]}
    
    for periodo in periodos:
        dados_periodo = dados_completos[
            (dados_completos['Mes'] == mes) & 
            (dados_completos['Periodo_Categoria'] == periodo)
        ]
        
        if len(dados_periodo) > 0:
            soma_pessoas = dados_periodo['Soma_Pessoas'].values[0]
        else:
            soma_pessoas = 0
        
        media_diaria = soma_pessoas / dias if dias > 0 else 0
        linha[periodo] = round(media_diaria, 2)
    
    linhas_grafico.append(linha)

df_grafico = pd.DataFrame(linhas_grafico)
print(f"✓ Aba 'Para Gráfico' criada com {len(df_grafico)} linhas")

# %% [markdown]
# # 16. Exportar para Excel

# %%
# Criar nome do arquivo com as datas do período
data_inicio_str = data_inicio.strftime('%Y%m%d')
data_fim_str = data_fim.strftime('%Y%m%d')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
nome_arquivo_saida = f"media_pessoas_aglomeracoes_{data_inicio_str}_a_{data_fim_str}_{timestamp}.xlsx"
caminho_saida = DOCS_DIR / nome_arquivo_saida

print(f"\n💾 Exportando para Excel:")
print("-" * 80)

try:
    with pd.ExcelWriter(caminho_saida, engine='openpyxl') as writer:
        # Aba Geral
        df_geral.to_excel(writer, sheet_name='Geral', index=False)
        
        # Abas por período
        df_madrugada = dfs_periodos['Madrugada']
        df_manha = dfs_periodos['Manhã']
        df_tarde = dfs_periodos['Tarde']
        df_noite = dfs_periodos['Noite']
        
        df_madrugada.to_excel(writer, sheet_name='05h - Madrugada', index=False)
        df_manha.to_excel(writer, sheet_name='10h - Manhã', index=False)
        df_tarde.to_excel(writer, sheet_name='15h - Tarde', index=False)
        df_noite.to_excel(writer, sheet_name='20h - Noite', index=False)
        
        # Aba para gráfico
        df_grafico.to_excel(writer, sheet_name='Para Gráfico', index=False)
        
        # Ajustar largura das colunas em todas as abas
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = min(max_length + 2, 50)
                worksheet.column_dimensions[column].width = adjusted_width
    
    print(f"✓ Arquivo exportado com sucesso!")
    print(f"✓ Localização: {caminho_saida}")
    print(f"✓ Nome do arquivo: {nome_arquivo_saida}")
    
except Exception as e:
    print(f"\n⚠️  ERRO ao exportar arquivo: {e}")

# %% [markdown]
# # 17. Resumo Executivo

# %%
print(f"\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)

print(f"\n📊 Análise Concluída:")
print(f"  • Período: {data_inicio.strftime('%d/%m/%Y')} até {data_fim.strftime('%d/%m/%Y')}")
print(f"  • Duração: {(data_fim - data_inicio).days + 1} dias")
print(f"  • Total de registros: {len(df_filtrado):,}")
print(f"  • Total de aglomerações (>10): {len(df_aglomeracoes):,}")
print(f"  • Meses analisados: {len(meses_unicos)}")

print(f"\n📁 Arquivo gerado:")
print(f"  • {nome_arquivo_saida}")
print(f"  • Localização: docs/")

print(f"\n📋 Abas criadas:")
print(f"  ✓ Geral - Visão consolidada de todos os períodos")
print(f"  ✓ 05h - Madrugada - Dados específicos da madrugada")
print(f"  ✓ 10h - Manhã - Dados específicos da manhã")
print(f"  ✓ 15h - Tarde - Dados específicos da tarde")
print(f"  ✓ 20h - Noite - Dados específicos da noite")
print(f"  ✓ Para Gráfico - Formatação ideal para criar gráficos")

print(f"\n✅ Análise concluída com sucesso!")
print("=" * 80)

ANÁLISE: Média de Pessoas em Aglomerações por Período
✓ Bibliotecas importadas
✓ Análise iniciada em: 12/11/2025 15:14:00

✓ Diretório base: c:\Users\x387162\Desktop\Relatorio_Diario\projeto_etl_dados-main
✓ Planilhas processadas: c:\Users\x387162\Desktop\Relatorio_Diario\projeto_etl_dados-main\data\processed
✓ Saída de documentos: c:\Users\x387162\Desktop\Relatorio_Diario\projeto_etl_dados-main\docs

📁 Arquivos disponíveis em 'data/processed/':
--------------------------------------------------------------------------------
  [1] Contagem_diaria_centro - Padronizada_processada_20251112_151254.xlsx
  [2] cruzamento_nomes_base_okuhara.xlsx
  [3] cruzamento_nomes_base_tablet.xlsx
  [4] lista_logradouros_corretos.xlsx
--------------------------------------------------------------------------------

✓ Arquivo selecionado: Contagem_diaria_centro - Padronizada_processada_20251112_151254.xlsx

DEFINIR PERÍODO DE ANÁLISE
✓ Data início definida: 01/06/2025
✓ Data fim definida: 11/11/2025

✓ Per